In [6]:
import pandas as pd
import numpy as np
from lenskit.algorithms import Recommender
from lenskit.algorithms.user_knn import UserUser
from lenskit.algorithms import user_knn
from lenskit.algorithms import Recommender
import warnings
warnings.filterwarnings('ignore') # supress warning instead of printing them every time



In [7]:
# Get the train and test data
data_train = pd.read_csv("../dataset/clean/user_ratings_neg_1000_20_20_1_train.csv")
data_train = data_train.drop(columns=['Unnamed: 0'])
display(data_train)

data_pop = pd.read_csv("../popular_jobs_full.csv")
users = data_train.drop_duplicates()
users = users["UserID"]
data_pop = data_pop[data_pop['UserId'].isin(users)]
display(data_pop)


,UserID,JobID,Rating
0,554,640492,1
1,554,280275,1
2,554,747584,1
3,554,25820,1
4,554,600058,1
...,...,...,...
241844,1470416,1103712,1
241845,1470416,338735,1
241846,1470416,1042356,1
241847,1471233,1114014,-1


,UserId,JobIds
21,554,747584 687730 439600 87580 789458 687729 67861...
32,769,340536 305416 28811 970973 908081 765210 54565...
61,1697,911255 601021 272504 512686 812531 314425 5842...
100,2652,617562 862785 1090398 567291 229326 17456 1084...
180,4359,281565 1093215 948355 820820 959921 300053 900...
...,...,...
389574,1467940,877091 1105002 797470 818055 214329 708809 684...
389576,1467962,374499 484960 1027092 884399 361843 254384 775...
389603,1468595,848076 384437 956125 548442 56192 500727 87709...
389645,1470416,384431 579410 974985 976980 827591 926897 7459...


In [10]:
data_pop = data_pop.assign(JobIds=data_pop['JobIds'].str.split(' ')).explode('JobIds', ignore_index=True)
display(data_pop)

,UserId,JobIds
0,554,747584
1,554,687730
2,554,439600
3,554,87580
4,554,789458
...,...,...
1636448,1471233,1114014
1636449,1471233,1104860
1636450,1471233,1104478
1636451,1471233,1103925


In [29]:
data_train = data_train[data_train["Rating"]==1]
display(data_train.head(20))


# remove rows if user applied
users = data_train["UserID"].drop_duplicates().tolist()

data_final = pd.DataFrame(columns=data_pop.columns.tolist())
for user in users[:10]:
    applied_jobs = data_train[data_train["UserID"]==user]['JobID'].astype(str).values.tolist()
    data_pop_temp = data_pop[data_pop["UserId"]==user]
    data_pop_temp = data_pop_temp[~data_pop_temp['JobIds'].isin(applied_jobs)]
    data_final = pd.concat([data_final, data_pop_temp])


display(data_final.head(20))


,UserID,JobID,Rating
0,554,640492,1
1,554,280275,1
2,554,747584,1
3,554,25820,1
4,554,600058,1
5,554,196603,1
6,554,1113088,1
7,554,40,1
8,554,627377,1
9,554,957,1


,UserId,JobIds
1,554,687730
2,554,439600
3,554,87580
4,554,789458
5,554,687729
6,554,678611
7,554,1113651
8,554,798756
9,554,300017
10,554,1042646


In [31]:
# Compute the user list to iterate through
all_users = data_pop["UserId"].drop_duplicates()

In [32]:
# get data_test
data_test = pd.read_csv("../dataset/clean/user_ratings_neg_1000_20_20_1_test.csv")
data_test = data_test.drop(columns=['Unnamed: 0'])

In [55]:
# Create iteration loop for evaluation

hitRate = 0.0 # counting hit rate
testUserNum = 10.0 # number of test users to compute hit rate
counter = 0

for idx, user in all_users.items():

    # keep track of how many users togo through
    counter+=1
    if (counter > testUserNum): break

    # compute test instances
    testInstances = data_test[data_test["UserID"]==user]

    # Create recommendations
    selected_jobs = data_final[data_final["UserId"]==user] 
    selected_jobs = selected_jobs.head(100)
    selected_jobs = selected_jobs.astype(np.int64)
    
    # check if any of the jobs is contained inside
    for idx in testInstances.index:
        job = testInstances.loc[idx, "JobID"]
        # count one hit if job is in the list
        if (selected_jobs["JobIds"] == job).any():
            hitRate+=1
            break 
    
print(hitRate/testUserNum)


,UserId,JobIds
150,769,340536
151,769,305416
152,769,28811
153,769,970973
154,769,908081
...,...,...
261,769,381978
262,769,995495
263,769,811914
264,769,551465


551465



,UserId,JobIds
300,1697,911255
306,1697,584241
307,1697,338167
308,1697,1045358
310,1697,1032353
...,...,...
416,1697,584887
417,1697,497411
418,1697,180076
419,1697,1101323


1045358



,UserId,JobIds
752,4743,628696
754,4743,451071
755,4743,246606
756,4743,637025
757,4743,846134
...,...,...
858,4743,474795
859,4743,474793
860,4743,466581
861,4743,381691


846109



,UserId,JobIds
900,4972,98665
901,4972,10312
902,4972,246611
903,4972,187358
904,4972,741664
...,...,...
1002,4972,854442
1003,4972,807810
1004,4972,677285
1005,4972,468908


218261



,UserId,JobIds
1051,6246,303721
1052,6246,88019
1053,6246,855952
1054,6246,1103747
1055,6246,834593
...,...,...
1176,6246,843399
1177,6246,460488
1178,6246,271082
1179,6246,35192


170621

0.5
